# The Big Mac Index

> "All I know is that when I needed McDonald's, McDonald's was there for me" **James Franco**



![image.png](https://dynamicmedia.zuza.com/zz/m/original_/b/e/be922a63-e0bf-49e2-99b3-f7acdc1f6eb2/McDburgers___Super_Portrait.jpg)

# 1. Purchasing Power Parity

One popular macroeconomic analysis metric to **compare** economic productivity and **standards of living between countries** is purchasing power parity (PPP). PPP is an economic theory that compares different countries' currencies through a "basket of goods" approach.

According to this concept, two currencies are in equilibrium—known as the currencies being at par—when a basket of goods is priced the same in both countries, taking into account the exchange rates

To measure the exchange rate between two currencies: 

$$ S = P_1  / P_2 $$ 

- \\( S \\) = Exchange rate of currency 1 to currency 2 
- \\( P_1 \\) = Cost of good X in currency 1
- \\( P_2 \\) = Cost of good X in currency 2



***

# 2. The Big Mac Index

Measuring Purchasing Power Parity between two currencies around the world is a tedious job, but luckily since 1986, **The Economist** created an attractive Index for this purpose. This index takes its name from the **Big Mac**, the famous hamburger sold at **McDonald's** restaurants around the world. It provides a test of the extent to which market exchange rates result in goods costing the same in different countries. 

The Big Mac PPP exchange rate between two countries is obtained by dividing the price of a Big Mac in one country (in its currency) by the price of a Big Mac in another country (in its currency). This value is then compared with the actual exchange rate; if it is lower, then the first currency is under-valued (according to PPP theory) compared with the second, and conversely, if it is higher, then the first currency is over-valued.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.graph_objects as go
from IPython.display import Image
from sklearn.linear_model import LinearRegression
from scipy import stats

We only calculate the Big Mac index for certain countries, specifically these ones:

In [ ]:
big_mac_countries = ['ARG', 'AUS', 'BRA', 'GBR', 'CAN', 'CHL', 'CHN', 'CZE', 'DNK',
                      'EGY', 'HKG', 'HUN', 'IDN', 'ISR', 'JPN', 'MYS', 'MEX', 'NZL',
                      'NOR', 'PER', 'PHL', 'POL', 'RUS', 'SAU', 'SGP', 'ZAF', 'KOR',
                      'SWE', 'CHE', 'TWN', 'THA', 'TUR', 'ARE', 'USA', 'COL', 'CRI',
                      'PAK', 'LKA', 'UKR', 'URY', 'IND', 'VNM', 'GTM', 'HND',
                      'NIC', 'AZE', 'BHR', 'HRV', 'JOR', 'KWT', 'LBN', 'MDA', 'OMN',
                      'QAT', 'ROU', 'EUZ']

The dataset, compiled every six months, contains three sets of data: the local price of a Big Mac, the exchange rate of the local currency to the US dollar, and the GDP per person of the country (in US dollars).

In [ ]:
big_mac = pd.read_csv('../input/big-mac-source-data.csv')
big_mac.tail()

In [ ]:
big_mac = big_mac[big_mac["iso_a3"].isin(big_mac_countries)]
big_mac.info()

## 2.2 Converting to a uniform currency

Our first step to calculate the index is to convert all of the prices to a uniform currency (we use the US dollar).

In [ ]:
big_mac["dollar_price"] = big_mac["local_price"] / big_mac["dollar_ex"]
big_mac.tail()

## 2.3 Calculating the raw index

Now that we've done this, we can pick out our five 'base' currencies: the US dollar (USD), Euro (EUR), British pound (GBP), Japanese yen (JPY), and Chinese yuan (CNY).

In [ ]:
base_currencies = ['USD', 'EUR', 'GBP', 'JPY', 'CNY']

Calculating the index is as simple as dividing the local price by the price in the base currency.

If a hamburger is selling in London for £2 and in New York for 4 $, this would imply a PPP exchange rate of 1 pound to 2 U.S. dollars. This PPP exchange rate may well be different from that prevailing in financial markets (so that the actual dollar cost of a hamburger in London may be either more or less than the 4 it sells for in New York)

In [ ]:
big_mac_2019 = big_mac[big_mac['date']==big_mac['date'].max()]

In [ ]:
for currency in base_currencies: 
    currency_price = float(big_mac_2019.loc[big_mac_2019["currency_code"]== currency, "dollar_price"])
    big_mac_2019[currency] = big_mac_2019["dollar_price"] / currency_price - 1

In [ ]:
big_mac_2019 = big_mac_2019.round(decimals = 3)
big_mac_2019.tail()

In [ ]:
fig = px.bar(big_mac_2019, x="USD", y="name", orientation='h', color="USD").update_yaxes(categoryorder="total ascending")
fig.show()

Countries with a positive value has an **overvalued exchange rate**, that  means goods are relatively more expensive in that country.

An overvalued exchange rate is particularly a problem during a period of sluggish growth. If the economy is booming, an overvalued exchange rate can help reduce inflationary pressure, but in a recession, an overvalued exchange rate can cause further deflationary pressures.

Countries with a negative value have an **undervalued currency**, wich means a currency with an exchange rate lower than it ought to be. A currency may be undervalued, for example, when its purchasing power, supply and demand are all strong, but its price is still comparatively low. 

Having a weaker currency relative to the rest of the world can help boost exports, shrink trade deficits and reduce the cost of interest payments on its outstanding government debts. There are, however, some negative effects of devaluations. They create uncertainty in global markets that can cause asset markets to fall or spur recessions. 

In [ ]:
fig = px.choropleth(big_mac_2019, locations="iso_a3",
                    color="USD", # lifeExp is a column of gapminder
                    hover_name="name", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

## 2.4 Calculating the adjusted index


While the Big Mac index is a refreshingly simple way of thinking about relative currency values, a common (and fair) objection to it is that burgers cannot be easily traded across borders. Given non-traded local inputs (rent and worker’s wages) one would expect Big Macs to be cheaper in poorer countries and dearer in wealthier ones.

We'll start out by only picking the countries where we have GDP data.

In [ ]:
big_mac_gdp_data = big_mac_2019[big_mac_2019["GDP_dollar"] > 0]
big_mac_gdp_data.tail()

In order to correct for the problem, we'll use a linear regression of GDP vs Big Mac Price.

We sometimes add or remove countries from the Big Mac index, but we want the list of countries on which we base the adjusted index to remain consistent. We use this list of countries to calculate the relationship between GDP and Big Mac price:

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(big_mac_gdp_data["GDP_dollar"], big_mac_gdp_data["dollar_price"])
# Create regression line
regressLine = intercept + big_mac_gdp_data["GDP_dollar"]*slope
plt.clf()

In [ ]:
fig = px.scatter(big_mac_gdp_data, x="GDP_dollar", y="dollar_price",
                 size='GDP_dollar', hover_data=['name'])
fig.add_trace(go.Line(y = regressLine, x =big_mac_gdp_data["GDP_dollar"], name = 'RegressLine'))

fig.update_layout(title='Big Mac prices v GDP per person',
                   xaxis_title='GDP per person ($)',
                   yaxis_title='Big Mac price ($)')

fig.show()

In [ ]:
big_mac_gdp_data["adj_price"] = regressLine
big_mac_adj_index = big_mac_gdp_data[["name","iso_a3","currency_code","dollar_price","adj_price","dollar_ex","GDP_dollar","date"]]
big_mac_adj_index.tail()

In [ ]:
for currency in base_currencies: 
    dollar_currency = float(big_mac_adj_index.loc[big_mac_adj_index["currency_code"]== currency, "dollar_price"])
    currency_price = float(big_mac_adj_index.loc[big_mac_adj_index["currency_code"]== currency, "adj_price"])
    big_mac_adj_index[currency] = (big_mac_adj_index["dollar_price"] / big_mac_adj_index["adj_price"]) / (dollar_currency / currency_price) - 1
big_mac_adj_index.tail()

In [ ]:
regression_countries = ['ARG', 'AUS', 'BRA', 'GBR', 'CAN', 'CHL', 'CHN', 'CZE', 'DNK',
                         'EGY', 'EUZ', 'HKG', 'HUN', 'IDN', 'ISR', 'JPN', 'MYS', 'MEX',
                         'NZL', 'NOR', 'PER', 'PHL', 'POL', 'RUS', 'SAU', 'SGP', 'ZAF',
                         'KOR', 'SWE', 'CHE', 'TWN', 'THA', 'TUR', 'USA', 'COL', 'PAK',
                         'IND', 'AUT', 'BEL', 'NLD', 'FIN', 'FRA', 'DEU', 'IRL', 'ITA',
                         'PRT', 'ESP', 'GRC', 'EST']

In [ ]:
big_mac_adj_index = big_mac_adj_index[big_mac_adj_index["iso_a3"].isin(regression_countries)]


In [ ]:
fig = px.bar(big_mac_adj_index, x="USD", y="name", orientation='h', color="USD").update_yaxes(categoryorder="total ascending")
fig.show()

# Drawbacks of Purchasing Power Parity

PPP is not a perfect score. The following factors explain why the purchasing power parity theory could not line up with reality.

- Transport Costs
- Tax Differences
- Government Intervention
- Non-Traded Services
- Market Competition



While it's not a perfect measurement metric, purchase power parity does let one compare pricing between countries. Just don't try to buy a hamburger in Switzerland if you plan on exchanging your money for Vietnamese dongs!